In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
pd.set_option("display.max_columns", None)
import json
import folium
import warnings
warnings.filterwarnings("ignore")
import requests
from selectolax.parser import HTMLParser
from pprint import pprint


In [12]:
mi_shops = pd.DataFrame(requests.post("https://www.michelin.com.tw/truck/api/getdealer.php", data={"category": "tryeplus"}).json()["data"])

In [86]:
res = requests.get("https://www.cst.com.tw/dealers.php?city=0&area=0&tire=32")
content = HTMLParser(res.text)
pages = int(content.css_first("ul.pagination > div > ul.pagination.pagination-lg > li:nth-last-child(2)").text())

In [98]:
dfs = []
for page in range(1, pages + 1):
    res = requests.get("https://www.cst.com.tw/dealers.php?city=0&area=0&tire=32&page={}".format(str(page)))
    content = HTMLParser(res.text)
    for c in content.css("div.row > div.col-md-6 > div.row > div.col-md-6:nth-last-child(1)"):
        s_name = c.css_first("h4").text()
        tel_number = c.css_first("p:nth-last-child(3)").text()
        address = c.css_first("p:nth-last-child(2)").text()
        data = [s_name, tel_number, address[:3], address]
        dfs.append(data)
    

In [99]:
maxxis = pd.DataFrame(dfs, columns=["name", "tel", "city", "address"])

In [100]:
maxxis

,name,tel,city,address
0,嘉鴻輪胎行,(04)27051041,台中市,台中市西屯區黎明路3段361號
1,正豐輪胎行,(06)3556060,台南市,台南市長和路一段3號
2,慶豐輪胎,(06)5933545,台南市,台南市安定區新吉里50號
3,登鉅汽車保修輪胎行,(03)5593546,新竹縣,新竹縣湖口鄉新興路336號
4,捷德輪胎精品企業有限公司,(04)24269689,台中市,台中市北屯區中清路二段899號
...,...,...,...,...
6,興源輪胎行,(04)25661355,台中市,台中市大雅區中清路三段1178號
7,信全輪胎行,(04)23357371,台中市,台中市烏日區太明北路1號
8,彰南輪胎行,04-8886470,彰化縣,彰化縣北斗鎮地政路346-1號
9,輪安輪胎行,(04)26350427,台中市,台中市龍井區沙田路四段171號


In [ ]:
def get_population():
    people = pd.read_csv("https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=CA18EE06-4A50-4861-9D97-7853353D7108", header=1) # 鄉鎮市區戶數、人口
    people = people.dropna()
    people = people[(~people["人口密度"].str.contains("…"))].reset_index(drop=True)
    people["城市"] = people["區域別"].map(lambda x: x[:3])
    people["鄉鎮市區"] = people["區域別"].map(lambda x: x[3:])
    geo = gpd.read_file("/Users/kai/Documents/playground_python/geo_pandas_practice/geo_datasets/mapdata202303230423/TOWN_MOI_1120317.shp", encoding="utf-8")
    result= pd.merge(people, geo, how="left", left_on=["城市", "鄉鎮市區"], right_on=["COUNTYNAME", "TOWNNAME"])
    result = result[['TOWNID', '城市', '鄉鎮市區', '年底人口數', '土地面積', '人口密度']]
    result = result.astype({"年底人口數": int, "人口密度": int})
    return result

In [ ]:
population = get_population()

In [ ]:
def get_geo_data():
    cc = gpd.read_file("/Users/kai/Documents/playground_python/geo_pandas_practice/geo_datasets/mapdata202303230423/TOWN_MOI_1120317.shp", encoding="utf-8") # 鄉鎮市區界圖(TWD97經緯度)
    cc = cc.to_crs({"init": "epsg:4326"})
    # cc_area = cc.copy()
    # cc_area = cc_area.to_crs(epsg=3826)
    # cc_area["area"] = round(cc_area["geometry"].area / 10 ** 6, 2)
    # cc["area"] = cc_area["area"]
    return cc

In [ ]:
tw_geo = get_geo_data()

In [ ]:
cc_geojson = json.loads(tw_geo.to_json())

In [ ]:
tw_geo

In [ ]:
ccToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "TOWNNAME", "TOWNENG"], ["縣市", "行政區", "ENG"])
map = folium.Map([23.973874611, 120.982024611], zoom_start=8)
folium.GeoJson(cc_geojson, name="基本資訊", tooltip=ccToolTip).add_to(map)
folium.Choropleth(
    geo_data = cc_geojson,
    name="各鄉鎮市區人口",
    data=population,
    columns=["TOWNID", "人口密度"],
    key_on="feature.properties.TOWNID",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="各鄉鎮市區人口密度",
).add_to(map)

folium.LayerControl().add_to(map)
map

In [ ]:
map.save("test.html")

In [ ]:
ccToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "TOWNNAME", "VILLNAME", "area"], ["縣市", "行政區", "村里", "面積"])

In [ ]:
y = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}
map = folium.Map([23.973874611, 120.982024611], zoom_start=8)
folium.GeoJson(cc_geojson, style_function=lambda x: y, tooltip=ccToolTip).add_to(map)

In [ ]:
map

In [ ]:
df = pd.read_excel("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/數據/Google地圖數據/google地圖輪胎店家數據.xlsx")

In [ ]:
geom = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=geom)

In [ ]:
gdf.crs = {'init' :'epsg:4326'}

In [ ]:
villiage = gpd.read_file("/Users/kai/Documents/playground_python/geo_pandas_practice/geo_datasets/mapdata202303230423/TOWN_MOI_1120317.shp", encoding="utf-8")
villiage = villiage.to_crs({"init": "epsg:4326"})

In [ ]:
# 計算polygon的面積需要將geometry轉為projection of Cartesian system，所以需要將geometry使用to_crs({'init': 'epsg:3857'})
villiage_area = villiage.copy()
villiage_area = villiage_area.to_crs({"init": "epsg:3826"})
villiage_area["area"] = round(villiage_area["geometry"].area / 10 ** 6, 2)

In [ ]:
villiage["area"] = villiage_area["area"]

In [ ]:
# villiage_geojson = json.loads(villiage[villiage["COUNTYNAME"] == "臺北市"].to_json())
villiage_geojson = json.loads(villiage.to_json())

In [ ]:
villToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "TOWNNAME", "area"], ["縣市", "行政區", "面積"])

In [ ]:
myMap = folium.Map([22.73444963475145, 120.28458595275877], zoom_start=14)
kaovilliage = folium.GeoJson(villiage_geojson, tooltip=villToolTip).add_to(myMap)
myMap.fit_bounds(kaovilliage.get_bounds())
myMap

In [ ]:
result = gpd.tools.sjoin(gdf, villiage[["COUNTYNAME", "TOWNNAME", "geometry"]], how="left")

In [ ]:
result_df = result.groupby(["COUNTYNAME", "TOWNNAME"]).size().reset_index(name="count")

In [ ]:
result.plot("COUNTYNAME")

In [ ]:
xxx = gdf[gdf["state"] == "台北市"]

In [ ]:
m = folium.Map((25.0133904,121.52245), tiles="Cartodb Positron", zoom_start=11)

In [ ]:
taipei = folium.GeoJson(villiage_geojson, tooltip=villToolTip)

In [ ]:
taipei.add_to(m)

In [ ]:
feature_group = folium.FeatureGroup("markers")

In [ ]:
json.loads(xxx.to_json())["features"][0]["properties"]

In [ ]:
for i in json.loads(xxx.to_json())["features"]:
    if "MICHELIN" in i["properties"]["name"]:
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="blue"), 
                      popup="Michelin").add_to(feature_group)
    elif "普利司通" in i["properties"]["name"]:
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="black"), 
                      popup="Bridgestone").add_to(feature_group)
    else:
        iframe = folium.IFrame("店名:" + str(i["properties"]["name"]) + "<br>" + "地址:" + str(i["properties"]["street"]) + "<br>" + "電話:" + str(i["properties"]["phone"]))
        popup = folium.Popup(iframe, min_width=150, max_width=200)
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="red"), 
                      popup=popup).add_to(feature_group)

In [ ]:
feature_group.add_to(m)

In [ ]:
m.save("test.html")

In [ ]:
# 綜合所得稅
tax = pd.read_csv("https://eip.fia.gov.tw/data/ias/ias109/109_165-9.csv")


In [ ]:
tax["城市"] = tax["鄉鎮市區"].map(lambda x: x[:3])
tax["鄉鎮市區"] = tax["鄉鎮市區"].map(lambda x: x[3:])

In [ ]:
tax_group = tax[(~tax["村里"].isin(["合計", "其他"]))].groupby(["城市", "鄉鎮市區", "村里"]).agg({"綜合所得總額": "sum", "納稅單位(戶)": "sum"}).reset_index()

In [ ]:
tax_group["平均所得"] = (tax_group["綜合所得總額"] /tax_group["納稅單位(戶)"]) * 1000

In [ ]:
tax_group["平均所得"] = tax_group["平均所得"].fillna(0).astype(int)

In [ ]:
tax_group.sort_values("平均所得", ascending=False)

In [ ]:
people.head(3)

In [ ]:
result = pd.merge(tax_group, people, how="left", left_on=["城市", "鄉鎮市區", "村里"], right_on=["COUNTYNAME", "TOWNNAME", "村里名稱"]).drop(["COUNTYNAME", "TOWNNAME", "村里名稱"], axis=1)

In [ ]:
result = result.fillna(0)

In [ ]:
result["納稅比"] = round(result["納稅單位(戶)"] / result["戶數"], 2)

In [ ]:
result.sort_values(by=["平均所得", "綜合所得總額", "納稅比"], ascending=False)

In [ ]:
final = pd.merge(cc, result, how="left", left_on=["COUNTYNAME", "TOWNNAME", "VILLNAME"], right_on=["城市", "鄉鎮市區", "村里"])

In [ ]:
final[["綜合所得總額", "納稅單位(戶)", "平均所得", "戶數", "人口數_合計", "人口數_男", "人口數_女"]] = final[["綜合所得總額", "納稅單位(戶)", "平均所得", "戶數", "人口數_合計", "人口數_男", "人口數_女"]].fillna(0)

In [ ]:
final = final.astype({"綜合所得總額": int, "納稅單位(戶)": int, "平均所得": int, "戶數": int, "人口數_合計": int, "人口數_男": int, "人口數_女": int})

In [ ]:
final["人口密度"] = round(final["人口數_合計"] / final["area"], 0)

In [ ]:
final